## Create dataset

## Instructions:
create testing folder in home (~/) directory 

download glove embeddings and place "glove.6B" folder in home~/ directory http://nlp.stanford.edu/data/glove.6B.zip

also, increase the speed of your instance (i used 16 CPUs and it took 6 hours to train 1 epoch of 3,000 word sequences)

In [1]:
# !pip install keras
# !pip install tensorflow
# !pip install plotly
# !pip install gensim
# !pip install Word2Vec
# !pip install get_tmpfile
# !pip install gensim.test.utils
# !pip install boto
# !pip install google-compute-engine



In [2]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.preprocessing.text import text_to_word_sequence






## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt
# import matplotlib.pyplot

# NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer


# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# from numpy import util


# Word2Vec
from gensim.test.utils import common_texts, get_tmpfile # not working
from gensim.models import Word2Vec

import re
from collections import Counter
import os









/home/raymondleemids/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/raymondleemids/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## [Skip this section] Split Dataset Train/Test 

In [3]:
#load and read final.csv
# final_csv.to_csv('final.csv')
# final_csv = pd.read_csv('final.csv')

In [4]:
# final_csv[:5]

In [5]:
### loaded into csv
# # convert integers to dummy variables (i.e. one hot encoded)
# encoder = LabelBinarizer()
# sparse_labels = encoder.fit_transform(final_csv['stock_overnight_movement_tag'])

In [6]:
### loaded into csv
# encoder.classes_

In [7]:
### loaded into csv
# # create sparse label columns
# sparse_labels = pd.DataFrame(data=sparse_labels,columns=['down', 'stay', 'up'])

In [8]:
### loaded into csv
# final_csv['down'] = sparse_labels['down']
# final_csv['stay'] = sparse_labels['stay']
# final_csv['up'] = sparse_labels['up']


In [9]:
# randomize dataset order 
# final_csv = shuffle(final_csv)

In [10]:
# # output shuffled.csv
# final_csv.to_csv('final_shuffled.csv')

## Load final_shuffled.csv

In [193]:
# load final_shuffled.csv
final_csv = pd.read_csv('final_shuffled.csv')

In [194]:
final_csv = final_csv.iloc[:1000]

In [195]:
# # filter final_csv to only include 'out of market'
# backup_csv = final_csv # backup in case i want to revert back
# final_csv = final_csv[final_csv['Time_of_day'] == 'out_of_market']

In [196]:
down_csv = pd.concat([final_csv[final_csv['stock_overnight_movement_tag'] == 'down'],
                      final_csv[final_csv['stock_overnight_movement_tag'] == 'stay'],
                      final_csv[final_csv['stock_overnight_movement_tag'] == 'up'].iloc[:300]])

In [197]:
down_csv

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,8K_Content,Adj Close_GSPC,Adj Close_stock,Close_GSPC,Close_stock,Date,Day Change_GSPC,...,Time,Time_of_day,Volume_GSPC,Volume_stock,Year,stock_overnight_movement_tag,8K_Content_cleaned,down,stay,up
2,71040,71317,112681,Check the appropriate box below ...,1372.18,9.10,1372.18,9.10,2012-02-28,4.62,...,2.012023e+13,out_of_market,3.579120e+09,4461800.0,2012.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
4,71914,72192,113869,Check the appropriate box below if the Form 8...,1460.91,24.93,1460.91,24.95,2012-10-17,6.69,...,2.012102e+13,out_of_market,3.655320e+09,10950000.0,2012.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
59,68493,68769,108833,Check the appropriate box below if the Form ...,1218.28,39.28,1218.28,40.04,2011-11-01,-32.72,...,2.011110e+13,out_of_market,5.645540e+09,2762300.0,2011.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
75,63514,63779,100706,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,1472.29,20.56,1472.29,20.56,2007-09-05,-16.47,...,2.007091e+13,out_of_market,2.991600e+09,2981100.0,2007.0,down,"['united', 'states', 'securities', 'and', 'exc...",1,0,0
86,7726,7806,12665,Table of Contents Check the appropriate ...,866.23,7.02,866.23,7.02,2009-04-24,12.32,...,2.009042e+13,out_of_market,7.114440e+09,445500.0,2009.0,down,"['table', 'of', 'contents', 'check', 'the', 'a...",1,0,0
92,60250,60515,95814,Check the appropriate box below if the Form ...,1182.45,89.52,1182.45,89.52,2010-10-27,-1.39,...,2.010103e+13,out_of_market,4.335670e+09,1402500.0,2010.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
105,28409,28516,45113,Check the appropriate box below if the For...,1261.01,38.71,1261.01,39.31,2011-12-07,2.87,...,2.011121e+13,out_of_market,4.160540e+09,579300.0,2011.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
110,51951,52178,82784,Check the appropriate box below if the Form ...,1319.44,40.55,1319.44,40.55,2011-03-29,10.07,...,2.011033e+13,out_of_market,3.482580e+09,9552200.0,2011.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
120,81621,81922,132708,Check the appropriate box below if the Form 8-...,1268.40,23.40,1268.40,23.40,2006-07-26,-0.47,...,2.006073e+13,out_of_market,2.667710e+09,4395900.0,2006.0,down,"['check', 'the', 'appropriate', 'box', 'below'...",1,0,0
138,33751,33894,53669,FORM 8-KCurrent Report Pursuant to Section 1...,1502.39,39.60,1502.39,41.79,2007-05-03,6.83,...,2.007050e+13,out_of_market,3.007970e+09,9729000.0,2007.0,down,"['form', '8', 'kcurrent', 'report', 'pursuant'...",1,0,0


In [198]:
# # # downsample
# downsamp_csv = pd.concat([final_csv[final_csv['down'] == 1], 
#      final_csv[final_csv['stay'] == 1].iloc[:300], 
#     final_csv[final_csv['up'] == 1]])

In [199]:
# downsamp_csv[downsamp_csv['down'] == 1].shape

In [200]:
# final_csv[]

In [201]:
# final_csv[final_csv['down'] == 1].shape

In [202]:
# final_csv[final_csv['down'] == 1].shape

In [203]:
# final_csv[final_csv['up'] == 1].shape

In [204]:
# downsamp_csv[downsamp_csv['up'] == 1].shape

In [205]:
# downsamp_csv[downsamp_csv['stay'] == 1].iloc[:300].shape

In [206]:
# final_csv[final_csv['stay'] == 1].iloc[:300].shape

In [207]:
# final_csv = pd.concat([final_csv[final_csv['down'] == 1], 
#      final_csv[final_csv['stay'] == 1].iloc[:300], 
#     final_csv[final_csv['up'] == 1]])

In [208]:
# final_csv.shape

In [209]:
# final_csv[final_csv['stay'] == 1].iloc[:300].shape

In [210]:
# final_csv

In [211]:
down_csv.shape

(1000, 33)

In [212]:
final_csv = down_csv

In [213]:
# final_csv.to_csv('final_downsampled.csv')

In [214]:
# final_csv = pd.read_csv('final_downsampled.csv')

## Create '8K_Content' sequences

In [215]:
# tokenize text. update this when you update sequence length

tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_csv['8K_Content_cleaned'])

sequences = tokenizer.texts_to_sequences(final_csv['8K_Content_cleaned'])
data = pad_sequences(sequences, maxlen=400) # takes about 5-10 mins

In [216]:
# vocab size
vocabulary_size = max(tokenizer.word_index.values())
vocabulary_size

44077

In [217]:
final_csv['8K_Content_sequences'] = data.tolist()

In [218]:
final_csv.shape

(1000, 34)

In [219]:
final_csv.shape

(1000, 34)

In [220]:
# sum(final_csv['down'])

In [221]:
# sum(final_csv['stay']) # there are 85,956 'stay' classes. will modify this to 10,750 to match down and up

In [222]:
# sum(final_csv['up'])

In [223]:
# final_csv[final_csv['stay'] == 1][:10750]

In [224]:
# downsample_csv = pd.concat([final_csv[final_csv['down'] == 1], 
#      final_csv[final_csv['stay'] == 1][:10750], 
#     final_csv[final_csv['up'] == 1]])

In [225]:
# sum(downsample_csv['down'])

In [226]:
# sum(downsample_csv['stay'])

In [227]:
# sum(downsample_csv['up'])

In [228]:
# 02-08 train. 09-10 develoment, 11-12 test
# X_train = final_csv['8K_Content_cleaned'][final_csv['Year'] <= 2008]
# X_dev = final_csv['8K_Content_cleaned'][(final_csv['Year'] >= 2009) & 
#                                        (final_csv['Year'] <= 2010)]
# X_test = final_csv['8K_Content_cleaned'][final_csv['Year'] >= 2011]

X_train = final_csv['8K_Content_sequences'][final_csv['Year'] <= 2008]
X_dev = final_csv['8K_Content_sequences'][(final_csv['Year'] >= 2009) & 
                                       (final_csv['Year'] <= 2010)]
X_test = final_csv['8K_Content_sequences'][final_csv['Year'] >= 2011]

y_train = final_csv[['down','stay','up']][final_csv['Year'] <= 2008]
y_dev = final_csv[['down','stay','up']][(final_csv['Year'] >= 2009) & 
                                       (final_csv['Year'] <= 2010)]
y_test = final_csv[['down','stay','up']][final_csv['Year'] >= 2011]

## Build and Train LSTM Model

In [229]:
# TODO: add Tensorboard

# reload(rnnlm)

# TF_GRAPHDIR = "/tmp/w266_project/finance_text_graph"

# # Clear old log directory.
# shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

# lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
# lm.BuildCoreGraph()
# lm.BuildTrainGraph()
# lm.BuildSamplerGraph()

# summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)


In [230]:
# TODO: update arbitary parameters, such as input_length
# build network

model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size, 100, input_length=400))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(3, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [231]:
# np.array(X_train[:5].values.tolist()).shape

In [232]:
# np.array(y_train[:5].values.tolist()).shape

In [233]:
# y_train[:5].values.shape

In [234]:
# np.array(y_train[:5].values).shape

In [235]:
np.array(X_dev.values.tolist()).shape

(216, 400)

In [236]:
np.array(y_train.values.tolist()).shape

(532, 3)

In [237]:
# train network

# model_lstm.fit(np.array(X_train[:5].values.tolist()), np.array(y_train[:5].values.tolist()), validation_split=0.4, epochs=5)

model_lstm.fit(np.array(X_train.values.tolist()), 
               np.array(y_train.values.tolist()), 
               validation_data=(np.array(X_dev.values.tolist()), 
                                np.array(y_dev.values.tolist())), 
               epochs=5)

#TODO: change validation split to validation_data

# takes about 3 mins per epoch using 2/3 of full dataset
# takes about 3 hours per epoch using full dataset (less testing set) 


Train on 532 samples, validate on 216 samples
Epoch 1/5
532/532 [==============================] - 21s 40ms/step - loss: 0.9063 - acc: 0.7857 - val_loss: 0.6811 - val_acc: 0.7685
Epoch 2/5
532/532 [==============================] - 20s 38ms/step - loss: 0.5800 - acc: 0.8177 - val_loss: 0.6844 - val_acc: 0.7685
Epoch 3/5
532/532 [==============================] - 21s 39ms/step - loss: 0.5162 - acc: 0.8177 - val_loss: 0.6575 - val_acc: 0.7685
Epoch 4/5
532/532 [==============================] - 18s 35ms/step - loss: 0.4393 - acc: 0.8289 - val_loss: 0.6681 - val_acc: 0.7685
Epoch 5/5
532/532 [==============================] - 19s 35ms/step - loss: 0.3522 - acc: 0.8797 - val_loss: 0.7785 - val_acc: 0.7731


In [238]:
# test network

model_lstm.evaluate(np.array(X_test.values.tolist()), 
                    np.array(y_test.values.tolist()))
                    

252/252 [==============================] - 1s 4ms/step


[0.7012252608935038, 0.7777777780143041]

In [239]:
# predict
model_lstm_preds = model_lstm.predict(np.array(X_test.values.tolist()))

In [ ]:
# display prediction probs
# model_lstm_preds[:5]

In [ ]:
# predictions to csv
np.savetxt('predictions_lstm_2.csv', model_lstm_preds, delimiter=',')
print("Saved predictions to disk: ", 'predictions_lstm_2.csv')

In [ ]:
# display prediction
np.argmax(model_lstm_preds, axis=1)

In [ ]:
# display actuals
np.argmax(np.array(y_test), axis=1)

In [ ]:
# prediction counts ['down','stay','up']
np.bincount(np.argmax(model_lstm_preds, axis=1))

In [ ]:
# actual counts ['down', stay', 'up']
np.bincount(np.argmax(np.array(y_test), axis=1))

In [ ]:
# save model

# serialize model to JSON
model_lstm_json = model_lstm.to_json()
with open("model_lstm_2.json", "w") as json_file:
    json_file.write(model_lstm_json)
# serialize weights to HDF5
model_lstm.save_weights("model_lstm_2.h5")
print("Saved model to disk: ", 'model_lstm_2.json and model_lstm_2.h5')

In [ ]:
# load model
# note: latest file updated 11/30. 3,000 word sequence.

# load json and create model
json_file = open('model_lstm_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_lstm_2.h5")
print("Loaded model from disk: ", 'model_lstm_2.json and model_lstm_2.h5')

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(np.array(X_test.values.tolist()), 
                              np.array(y_test.values.tolist()), verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
# loaded_model.predict(X_test)

## LSTM with 1D Convolutional Layer

In [ ]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=400))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(3, activation='softmax'))
    model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [ ]:
model_conv = create_conv_model()
model_conv.fit(np.array(X_train.values.tolist()), 
               np.array(y_train.values.tolist()), 
               validation_data=(np.array(X_dev.values.tolist()), 
                                np.array(y_dev.values.tolist())), 
               epochs=5)

In [ ]:
model_conv.evaluate(np.array(X_train.values.tolist()), 
               np.array(y_train.values.tolist()))

In [ ]:
model_conv_preds = model_conv.predict(np.array(X_test.values.tolist()))

In [ ]:
# save predictions to csv
np.savetxt('predictions_conv_2.csv', model_conv_preds, delimiter=',')
print("Saved predictions to disk: ", 'predictions_conv_2.csv')

In [ ]:
# # save model

model_conv_json = model_conv.to_json()
with open("model_conv_2.json", "w") as json_file:
    json_file.write(model_conv_json)
# serialize weights to HDF5
model_conv.save_weights("model_conv_2.h5")
print("Saved model to disk:", 'model_conv_2.json and model_conv_2.h5')

In [ ]:
# load model

# load json and create model
json_file = open('model_conv_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_conv_2.h5")
print("Loaded model from disk: ", 'model_conv_2.json and model_conv_2.h5')

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(np.array(X_test.values.tolist()), 
                              np.array(y_test.values.tolist()), verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [77]:
# loaded_model.predict(X_test)

## Use pre-trained Glove word embeddings

In [ ]:
# # load GLOVE embeddings
# embeddings_index = dict()
# f = open('~/glove.6B/glove.6B.100d.txt')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# !cd ~/glove.6B

In [ ]:
# embeddings_index

In [ ]:
# # create a weight matrix for words in training docs
# embedding_matrix = np.zeros((vocabulary_size, 100))
# for word, index in tokenizer.word_index.items():
#     if index > vocabulary_size - 1:
#         break
#     else:
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None:
#             embedding_matrix[index] = embedding_vector

In [ ]:
# # LSTM and CNN model
# model_glove = Sequential()
# model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
# model_glove.add(Dropout(0.2))
# model_glove.add(Conv1D(64, 5, activation='relu'))
# model_glove.add(MaxPooling1D(pool_size=4))
# model_glove.add(LSTM(100))
# model_glove.add(Dense(1, activation='softmax'))
# model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# # train model
# model_glove.fit(data, merged_csv['movement'], validation_split=0.4, epochs=3)

In [ ]:
# # evaluate model
# model_glove.evaluate(data, merged_csv['movement'])

## Use Word2Vec to train word embeddings on corpus

In [ ]:
# from gensim.models import Word2Vec
# import nltk
# # nltk.download('punkt')

In [ ]:
# # tokenize financial data
# merged_csv['tokenized'] = merged_csv.apply(lambda row : nltk.word_tokenize(row['8K_Content_cleaned']), axis=1)


In [ ]:
# merged_csv.head()


In [ ]:
# #train
# model_w2v = Word2Vec(merged_csv['tokenized'], size=100)

In [ ]:
# X = model_w2v[model_w2v.wv.vocab]

In [ ]:
# X.shape

In [ ]:
# # Create LSTM and CNN model

# model_w2v = Sequential()
# model_w2v.add(Embedding(1925, 100, input_length=50, weights=[X], trainable=False))
# model_w2v.add(Dropout(0.2))
# model_w2v.add(Conv1D(64, 5, activation='relu'))
# model_w2v.add(MaxPooling1D(pool_size=4))
# model_w2v.add(LSTM(100))
# model_w2v.add(Dense(1, activation='softmax'))
# model_w2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# # train model
# model_w2v.fit(data, merged_csv['movement'], validation_split=0.4, epochs=3)


## Plot Word Vectors Using PCA

In [ ]:
# from sklearn.decomposition import TruncatedSVD


In [ ]:
# tsvd = TruncatedSVD(n_components=5, n_iter=10)
# result = tsvd.fit_transform(X)

In [ ]:
# result.shape

In [ ]:
# tsvd_word_list = []
# words = list(model_w2v.wv.vocab)
# for i, word in enumerate(words):
#     tsvd_word_list.append(word)

In [ ]:
# len(words)

In [ ]:
# tsvd_word_list = []
# words = list(model_w2v.wv.vocab)
# for i, word in enumerate(words):
#     tsvd_word_list.append(word)

# trace = go.Scatter(
#     x = result[0:len(words), 0], 
#     y = result[0:len(words), 1],
#     mode = 'markers',
#     text= tsvd_word_list[0:len(words)]
# )

# layout = dict(title= 'SVD 1 vs SVD 2',
#               yaxis = dict(title='SVD 2'),
#               xaxis = dict(title='SVD 1'),
#               hovermode= 'closest')

# fig = dict(data = [trace], layout= layout)
# py.iplot(fig)

## Visualize word embeddings

In [ ]:
# lstm_embds = model_lstm.layers[0].get_weights()[0]

In [ ]:
# conv_embds = model_conv.layers[0].get_weights()[0]

In [ ]:
# glove_emds = model_lstm.layers[0].get_weights()[0]

In [ ]:
# word_list = []
# for word, i in tokenizer.word_index.items():
#     word_list.append(word)

In [ ]:
# def plot_words(data, start, stop, step):
#     trace = go.Scatter(
#         x = data[start:stop:step,0], 
#         y = data[start:stop:step, 1],
#         mode = 'markers',
#         text= word_list[start:stop:step]
#     )
#     layout = dict(title= 't-SNE 1 vs t-SNE 2',
#                   yaxis = dict(title='t-SNE 2'),
#                   xaxis = dict(title='t-SNE 1'),
#                   hovermode= 'closest')
#     fig = dict(data = [trace], layout= layout)
#     py.iplot(fig)

In [ ]:
# # LSTM embeddings 

# lstm_tsne_embds = TSNE(n_components=2).fit_transform(lstm_embds)

In [ ]:
# plot_words(lstm_tsne_embds, 0, 2000, 1)


In [ ]:
# # CNN + LSTM
# conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)


In [ ]:
# plot_words(conv_tsne_embds, 0, 2000, 1)


In [ ]:
# # Glove

# glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)


In [ ]:
# plot_words(glove_tsne_embds, 0, 2000, 1)
